In [239]:
import psycopg2
from psycopg2.extras import Json, DictCursor
import json
import os
import glob
import gzip
import re
import unidecode
import unicodedata


In [240]:
## method block 1
def normalize_title(title_string):
    title_string_norm = re.sub('[^\w]', ' ', title_string)
    title_string_norm = re.sub('\s+', ' ', title_string_norm)
    title_string_norm = unicodedata.normalize('NFD', title_string_norm)
    title_string_norm = unidecode.unidecode(title_string_norm)
    return title_string_norm.strip().lower()


def normalize_author_name(author_string):
    author_string_norm = re.sub('[^\w]', ' ', author_string)
    author_string_norm = re.sub('\s+', ' ', author_string_norm)
    author_string_norm = unicodedata.normalize('NFD', author_string_norm)
    author_string_norm = unidecode.unidecode(author_string_norm)
    return author_string_norm.strip().lower()


id_keys_in_output = ['open_alex_id', 'sem_open_alex_id', 'pubmed_id', 'pmc_id', 'doi']


def map_ids_from_openalexdb_match_to_dict(matched_pub_from_db):
    openalexdb_match_ids = matched_pub_from_db[4]
    bib_entry_ids_dict = {}

    # go through all five ID types in openalexdb and add to temporary bib_entry_ids_dict
    # recall openalexdb IDs column structure: list of IDs [openalexid, semopenalexid, pubmedid, pmcid, doi]
    # recall wanted dict structure: id_keys_in_output = ['open_alex_id', 'sem_open_alex_id', 'pubmed_id', 'pmc_id', 'doi']

    for id_type_iterator in range(5):
        bib_entry_ids_dict[id_keys_in_output[id_type_iterator]] = openalexdb_match_ids[id_type_iterator]
    return bib_entry_ids_dict


def vary_title_window(normalized_title_string):
    title_tokenized = normalized_title_string.split(" ")
    title_omit_first_token = title_tokenized[1:]
    title_omit_first_token = " ".join(title_omit_first_token)
    title_omit_last_token = title_tokenized[:-1]
    title_omit_last_token = " ".join(title_omit_last_token)
    return title_omit_first_token, title_omit_last_token


In [241]:
## method block 2

def item_authors_in_ref_string(openalex_item_authors_list, ref_string):
    any_author_in_ref_string = False
    ref_string_normalized = normalize_author_name(ref_string)
    for openalex_item_author in openalex_item_authors_list:
        if openalex_item_author.split(" ")[-1] in ref_string_normalized.split(
                " "):  # check occurence of author in token-wise ref string
            print(openalex_item_author.split(" ")[-1])  # last name
            any_author_in_ref_string = True

    return any_author_in_ref_string


def match_title_in_openalexdb(query_string, bib_entry_title_norm, bib_entry_ref_string, try_title_windows_flag: False):
    cursor = conn.cursor()
    cursor.execute(query_string, (bib_entry_title_norm,))
    matching_openalex_pubs = cursor.fetchall()

    # if only one result is found, proceed to handover IDs from openalexdb
    if len(matching_openalex_pubs) == 1:
        print("Ein passender Eintrag in openalexdb gefunden - check if author in OpenAlex entry")

        # look for authors in bib_entry_ref_string
        openalex_item_authors_list = matching_openalex_pubs[0][2]
        if item_authors_in_ref_string(openalex_item_authors_list, bib_entry_ref_string):
            return matching_openalex_pubs[0]

        # author not found in refstring
        else:
            print('Authors maintained in OpenAlex for current item not present in ref string - no match - item skipped')
            return

    # more than one possible title match found in openalexdb
    elif len(matching_openalex_pubs) > 1:
        print(
            "Mehr als ein Eintrag mit passendem titel gefunden - select those with authors present in ref string - from these, start citation sort")

        matching_openalex_pubs_with_author_present = []
        for match in matching_openalex_pubs:
            openalex_item_authors_list = match[2]

            # consider only the title matched pubs that have an author match with the ref string for citation count comparison in the next step
            if item_authors_in_ref_string(openalex_item_authors_list, bib_entry_ref_string):
                matching_openalex_pubs_with_author_present.append(match)

        # iterate through returned matches to identify and select the most cited
        if len(matching_openalex_pubs_with_author_present) != 0:
            citation_counts_of_matches = []
            for match in matching_openalex_pubs_with_author_present:
                citation_counts_of_matches.append(match[3])

            index_of_most_cited_in_match_list = citation_counts_of_matches.index(max(citation_counts_of_matches))
            matched_openalex_pub = matching_openalex_pubs_with_author_present[index_of_most_cited_in_match_list]
            print(matching_openalex_pubs)
            return matched_openalex_pub

        else:
            print(
                'Authors maintained in OpenAlex for title matched items all not present in ref string - no matches - items skipped')
            return

    # no match found -> try alternate title substrings and redo search
    elif len(matching_openalex_pubs) == 0:
        if try_title_windows_flag:
            print("NIX gefunden - no match of title in OpenAlex - check different window of title")
            bib_entry_title_norm_omit_first, bib_entry_title_norm_omit_last = vary_title_window(bib_entry_title_norm)

            matched_openalex_pub = match_title_in_openalexdb(query_string, bib_entry_title_norm_omit_first,
                                                             bib_entry_ref_string, False)
            if matched_openalex_pub is not None:
                return matched_openalex_pub
            else:
                matched_openalex_pub = match_title_in_openalexdb(query_string, bib_entry_title_norm_omit_last,
                                                                 bib_entry_ref_string, False)
                if len(matched_openalex_pub) is not None:
                    return matched_openalex_pub
                else:
                    # occurs when no match is found for original, and both window versions of the bib_entry_title
                    return
        else:
            # occurs when calling this matching function with flag for trying different title windows = False (usually in 1st recursive call of function to avoid infinite call loop)
            print("NIX gefunden - no match of title in OpenAlex - check for different title windows SKIPPED")
            return

    else:
        print("Unhandled matching scenario occurred.")
        return


In [276]:
#establishing the connection
conn = psycopg2.connect(
    database="openalexdb", user='postgres', password='0000',
    host='127.0.0.1', port='5432'  #host=localhost
)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method for test
cursor.execute("select version()")

#Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ", data)


Connection established to:  ('PostgreSQL 15.1, compiled by Visual C++ build 1914, 64-bit',)


In [277]:
# remove previous instance of openalex table if existing

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS openalex")
conn.commit()

#cursor.execute("DROP TABLE IF EXISTS openalex")

conn = psycopg2.connect(database="openalexdb", user='postgres', password='0000', host='127.0.0.1', port='5432')
cursor = conn.cursor()
cursor.execute('''
        CREATE TABLE openalex (
            number SERIAL PRIMARY KEY,
            normalized_title VARCHAR,
            authors VARCHAR[],
            citation_count INTEGER,
            ids VARCHAR[]
        )
''')
conn.commit()

print("Table created successfully........")

Table created successfully........


In [274]:
# insert single line for test purposes

conn = psycopg2.connect(database="openalexdb", user='postgres', password='0000', host='127.0.0.1', port='5432')
cursor = conn.cursor()

json_raw = {'open_alex_id': 987, 'pmc_id': 123, 'pubmed_id': 555, 'doi': 'http://doiurl.com'}

# modify raw json dict of IDs to fit database format
s = ''
for v in json_raw.values():
    s += '"' + str(v) + '",'
s = "{" + str(s)[:-1] + "}"

# s is now: {"987","123","555","http://doiurl.com"}

a = '{"author1", "author2"}'

cursor.execute("INSERT INTO openalex (number, normalized_title, authors, citation_count, ids) VALUES (%s,%s,%s,%s,%s)",
               (4, 'Titel', a, 75, s))


# Commit changes in the database
conn.commit()
conn.close()

In [275]:
# retrieve the test line from above

conn = psycopg2.connect(database="openalexdb", user='postgres', password='0000', host='127.0.0.1', port='5432')
cursor = conn.cursor()

cursor.execute("SELECT * FROM openalex")
result = cursor.fetchone()
print(result)

conn.commit()
conn.close()

(4, 'Titel', ['author1', 'author2'], 75, ['987', '123', '555', 'http://doiurl.com'])


In [278]:
# load some OpenAlex data test wise

i = 0
error_count = 0
id_keys_in_db = ['open_alex_id', 'sem_open_alex_id', 'pubmed_id', 'pmc_id', 'doi']

openalex_data_files_dir = r'C:\Users\Johan\Documents\_Uni\MA\SemOpenAlex_Data\before_trafo\works\*'
for filename in glob.glob(os.path.join(openalex_data_files_dir, '*.gz')):
    with gzip.open(filename, 'r') as f:
        for line in f:
            try:
                i += 1
                json_data = json.loads(line.decode('utf-8'))

                # get data from current json line for current object

                work_title_orig = json_data['title']

                # normalize title
                if work_title_orig is not None:
                    # example orig title: Reversible Mechanochromic Luminescence of [(C6F5Au)2(μ-1,4-Diisocyanobenzene)]
                    # normalized title: reversible mechanochromic luminescence of c6f5au 2 m 1 4 diisocyanobenzene
                    work_title_norm = normalize_title(work_title_orig)

                else:
                    work_title_norm = ""

                work_authorships = json_data['authorships']
                work_author_list = []
                for author in work_authorships:
                    # normalize author name for every involved author
                    work_author_norm = normalize_author_name(author['author']['display_name'])
                    work_author_list.append(work_author_norm)

                # modify string so that it suits postgresql syntax to insert into table as a list of strings
                work_author_list_str = str(work_author_list).replace('[', '{').replace(']', '}').replace("'", '"')
                # example list str now:
                # {"engelbert knosp", "erich steiner", "klaus kitz", "christian matula"}

                work_cited_by_count = json_data['cited_by_count']

                ids = json_data.get('ids')
                # example ids list: {'openalex': 'https://openalex.org/W2169161843', 'doi': 'https://doi.org/10.1177/1362361312472989', 'pmid': 'https://pubmed.ncbi.nlm.nih.gov/23614935', 'mag': 2169161843}
                ids_list = []
                # wanted: openalex, semopenalex, pubmed, pmc, doi

                if not ids['openalex'] is None:
                    ids_list.append(ids['openalex'])
                    ids_list.append(ids['openalex'].replace("https://openalex.org", "https://semopenalex.org/work"))
                else:
                    ids_list.append("")
                    ids_list.append("")

                pmid = json_data.get('ids').get('pmid')
                if not pmid is None:
                    ids_list.append(pmid)
                else:
                    ids_list.append("")

                pmc_id = json_data.get('ids').get('pmcid')
                if not pmc_id is None:
                    ids_list.append(pmc_id)
                else:
                    ids_list.append("")

                doi = json_data['doi']
                if not doi is None:
                    ids_list.append(doi)
                else:
                    ids_list.append("")

                # adapt to postgresql syntax to insert into table as a list of strings
                ids_list = str(ids_list).replace('[', '{').replace(']', '}').replace("'", '"')

                # write data of current object to database table
                # write the values number, work_title_norm, work_author_list_str, work_cited_by_count, ids_list

                conn = psycopg2.connect(database="openalexdb", user='postgres', password='0000', host='127.0.0.1',
                                        port='5432')
                cursor = conn.cursor()
                cursor.execute(
                    "INSERT INTO openalex (number, normalized_title, authors, citation_count, ids) VALUES (%s,%s,%s,%s,%s)",
                    (i, work_title_norm, work_author_list_str, work_cited_by_count, ids_list))
                print(f"Inserting line {i} into table..")
                conn.commit()
                conn.close()

            except Exception as e:
                print(f'{e} error in line {i} of file {filename}')
                error_count += 1
                pass


Inserting line 1 into table..
Inserting line 2 into table..
Inserting line 3 into table..
Inserting line 4 into table..
Inserting line 5 into table..
Inserting line 6 into table..
Inserting line 7 into table..
Inserting line 8 into table..
Inserting line 9 into table..
Inserting line 10 into table..
Inserting line 11 into table..
Inserting line 12 into table..
Inserting line 13 into table..
Inserting line 14 into table..
Inserting line 15 into table..
Inserting line 16 into table..
Inserting line 17 into table..
Inserting line 18 into table..
Inserting line 19 into table..
Inserting line 20 into table..
Inserting line 21 into table..
Inserting line 22 into table..
Inserting line 23 into table..
Inserting line 24 into table..
Inserting line 25 into table..
Inserting line 26 into table..
Inserting line 27 into table..
Inserting line 28 into table..
Inserting line 29 into table..
Inserting line 30 into table..
Inserting line 31 into table..
Inserting line 32 into table..
Inserting line 33

KeyboardInterrupt: 

In [279]:
# iterate through latexparse output JSONL to get set of bib entries for every object
# determine title in bib entry ref strings (either through arXiv ID or DOI first, alternatively through GROBID API [all three not covered here])
# normalize this title and match it with OpenAlex data in openalexdb table (created above)
# use openalexdb table data to fill in IDs for the current bib entry item (openalex, semopenalex, pubmed, pmc, doi)

latexparse_ouput_dir = r'C:\Users\Johan\Documents\_Uni\Hiwi Tasks\LatexParse\local_latexparse-master\test_out'
id_keys_in_output = ['open_alex_id', 'sem_open_alex_id', 'pubmed_id', 'pmc_id', 'doi']

i = 0
error_count = 0
conn = psycopg2.connect(database="openalexdb", user='postgres', password='0000', host='127.0.0.1', port='5432')

for filename in glob.glob(os.path.join(latexparse_ouput_dir, '*.jsonl')):
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                json_data = json.loads(line)
                i += 1
                item_bib_entries = json_data['bib_entries']
                for bib_entry in item_bib_entries.keys():
                    ### title determination method goes here
                    ### pseudo: bib_entry_title = determine_title(item_bib_entries[bib_entry]['bib_entry_raw'])
                    # print(item_bib_entries[bib_entry]['bib_entry_raw'])
                    pass

                # artificial title example for lookup in openalexdb table
                if i == 1:  ##delete this condition once trials succeed
                    bib_entry_title = "SARS-CoV-2 Viral Load in Upper Respiratory Specimens of Infected Patients WORTZUVIEL."
                    #bib_entry_title = "Pituitary adenomas with invasion of the cavernous sinus space: a magnetic resonance imaging classification compared with surgical findings."
                    bib_entry_ref_string = "Knosp, E., He Steiner, E., Kitz, K. & Matula, C. Pituitary adenomas with invasion of the cavernous sinus space: a magnetic resonance imaging classification compared with surgical findings. Neurosurgery 33, 610–617 (1993)"
                    #bib_entry_ref_string = "Pituitary adenomas with invasion of the cavernous sinus space: a magnetic resonance imaging classification compared with surgical findings. Neurosurgery 33, 610–617 (1993)"
                    bib_entry_title_norm = normalize_title(bib_entry_title)
                    # bib_entry_title_norm now = "sars cov 2 viral load in upper respiratory specimens of infected patients"

                    bib_entry_ids_dict = {}

                    openalexdb_title_query = "SELECT * from openalex WHERE normalized_title=%s"

                    matching_openalex_pub = match_title_in_openalexdb(openalexdb_title_query, bib_entry_title_norm,
                                                                      bib_entry_ref_string, try_title_windows_flag=True)
                    if matching_openalex_pub is None:
                        print('no match was returned')
                        bib_entry_ids_dict = {'open_alex_id': '', 'sem_open_alex_id': '', 'pubmed_id': '', 'pmc_id': '',
                                              'doi': ''}
                    elif matching_openalex_pub is not None:
                        print('MATCH was returned')
                        print(matching_openalex_pub)
                        bib_entry_ids_dict = map_ids_from_openalexdb_match_to_dict(matching_openalex_pub)


                    # for analysis (delete later): print matches in db

                    print(bib_entry_ids_dict)
                    # structure of bib_entry_ids_dict now:
                    # {'open_alex_id': 'https://openalex.org/W3006961006', 'sem_open_alex_id': 'https://semopenalex.org/work/W3006961006', 'pubmed_id': 'https://pubmed.ncbi.nlm.nih.gov/32074444', 'pmc_id': 'https://www.ncbi.nlm.nih.gov/pmc/articles/7121626', 'doi': 'https://doi.org/10.1056/nejmc2001737'}



            except Exception as e:
                print(f'{e} error in line {i} of file {filename}')
                error_count += 1
                pass

NIX gefunden - no match of title in OpenAlex - check different window of title
NIX gefunden - no match of title in OpenAlex - check for different title windows SKIPPED
Ein passender Eintrag in openalexdb gefunden - check if author in OpenAlex entry
he
MATCH was returned
(18, 'sars cov 2 viral load in upper respiratory specimens of infected patients', ['lirong zou', 'feng ruan', 'mingxing huang', 'lijun liang', 'huitao huang', 'zhongsi hong', 'jianxiang yu', 'min kang', 'yingchao song', 'jinyu xia', 'qianfang guo', 'tie song', 'jianfeng he', 'hui ling yen', 'malik peiris', 'jie wu'], 3306, ['https://openalex.org/W3006961006', 'https://semopenalex.org/work/W3006961006', 'https://pubmed.ncbi.nlm.nih.gov/32074444', 'https://www.ncbi.nlm.nih.gov/pmc/articles/7121626', 'https://doi.org/10.1056/nejmc2001737'])
{'open_alex_id': 'https://openalex.org/W3006961006', 'sem_open_alex_id': 'https://semopenalex.org/work/W3006961006', 'pubmed_id': 'https://pubmed.ncbi.nlm.nih.gov/32074444', 'pmc_id': '